In [1]:
import torch
import sys
import os

# Get the absolute path to the project root & add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from pde_class import Pde
from simple_pinn import PINN
from helper_functions import gradient, plot_loss, getData


In [ ]:
pde = Pde(E=1e8, nu=0.3, h=0.001, k=5/6, l=1.0, q = 1.0)
model = PINN([1, 20, 20, 20, 4], lr=0.5e-3)

iterations = 100001
report_step = 1000
rel_diff = 100.0
for i in range(iterations):
      model.optimizer.zero_grad()

      # Generate N points
      x = getData(num_points=10, grad=True)
      resloss, derivloss = pde.getLoss(model, x)

      loss = resloss + derivloss
      loss.backward()

      model.optimizer.step()

      with torch.no_grad():
          if i % report_step ==0:
              print(f"iteration {i} resloss {resloss.item():.2e}")
              print(f"iteration {i} derivloss {derivloss.item():.2e}")
              plot_loss(model=model, eq=pde)
              print